## HW 6

## Team- E Navaneet Kumar, Rithvika Paladugu

In [2]:
import numpy as np
import csv

# Set up file paths
dataDir = 'C:/Users/navne/Python Files/Data Acquisition BIA 627/HW5/'
file_name_test = dataDir + 'testTrack_hierarchy.txt'
file_name_train = dataDir + 'trainIdx2_matrix.txt'
output_file = dataDir + 'output1.csv'  # Changed the extension to .csv

# Loading the training data into a dictionary
train_data = {}
with open(file_name_train, 'r') as fTrain:
    for line in fTrain:
        userID, itemID, rating = line.strip().split('|')[:3]
        if userID not in train_data:
            train_data[userID] = {}
        train_data[userID][itemID] = int(rating)

# then we process the test data and write predictions
with open(file_name_test, 'r') as fTest, open(output_file, 'w', newline='') as fOut:
    csv_writer = csv.writer(fOut)
    csv_writer.writerow(['TrackID', 'Predictor'])  # Into the csv file we will add the header

    lastUserID = None  # Track the last user ID processed
    trackID_vec = []  # Initialize the track ID vector for each user

    for line in fTest:
        arr_test = line.strip().split('|')
        if len(arr_test) < 4:
            continue  # we will skip if  then line doesn't have enough data
        userID, trackID, albumID, artistID = arr_test[:4]   # only considering these 4 entities 

        # Reset trackID_vec for a new user
        if userID != lastUserID:
            if lastUserID is not None:
                # Write predictions for the previous user's tracks
                for i, predRating in enumerate(predRatings):
                    trackID_pred = f"{lastUserID}_{trackID_vec[i]}"
                    csv_writer.writerow([trackID_pred, int(predRating > 0)]) 

            trackID_vec = [trackID]  # Start a new list for the new user
            predRatings = np.zeros(6)  # Reset predicted ratings for the new user
        else:
            trackID_vec.append(trackID)  # Append trackID for the same user

        # Check if the user exists in the training data
        if userID in train_data:
            user_ratings = train_data[userID]

            # Predict based on album or artist rating, if available
            if albumID in user_ratings:
                predRatings[len(trackID_vec) - 1] = user_ratings[albumID]
            elif artistID in user_ratings:
                predRatings[len(trackID_vec) - 1] = user_ratings[artistID]

        lastUserID = userID  # Update the last user ID processed

    # Write predictions for the last user in the file
    for i, predRating in enumerate(predRatings):
        trackID_pred = f"{userID}_{trackID_vec[i]}"
        csv_writer.writerow([trackID_pred, int(predRating > 0)])  # Write to CSV file


In [3]:
import numpy as np
import csv

def calculate_stats(ratings):
    if not ratings:
        return [0] * 6  # Return zeros if no ratings
    
    num_ratings = len(ratings)
    max_score = max(ratings, default=0)
    min_score = min(ratings, default=0)
    sum_score = sum(ratings)
    avg_score = sum_score / num_ratings if num_ratings > 0 else 0
    #var_score = np.var(ratings) if num_ratings > 0 else 0
    #return [num_ratings, max_score, min_score, sum_score, avg_score, var_score]
    std_dev = np.std(ratings) if ratings else 0
    return [num_ratings, max_score, min_score, sum_score, avg_score, std_dev]

    

dataDir = 'C:/Users/navne/Python Files/Data Acquisition BIA 627/HW5/'
file_name_test = dataDir + 'testTrack_hierarchy.txt'
file_name_train = dataDir + 'trainIdx2_matrix.txt'
output_file = dataDir + 'output2.csv'

train_data = {}
with open(file_name_train, 'r') as fTrain:
    for line in fTrain:
        userID, itemID, rating = line.strip().split('|')[:3]
        train_data.setdefault(userID, {})[itemID] = int(rating)

with open(file_name_test, 'r') as fTest, open(output_file, 'w', newline='') as fOut:
    csv_writer = csv.writer(fOut)
    csv_writer.writerow(['TrackID', 'Predictor'])

    for line in fTest:
        parts = line.strip().split('|')
        userID, trackID = parts[0], parts[1]
        kaggle_trackID = f"{userID}_{trackID}"

        albumID, artistID = parts[2], parts[3]
        genreIDs = parts[4:]

        # Initialize ratings list with album and artist ratings if they exist
        ratings = []
        if albumID in train_data.get(userID, {}):
            ratings.append(train_data[userID][albumID])
        if artistID in train_data.get(userID, {}):
            ratings.append(train_data[userID][artistID])

        # Append genre ratings to the ratings list
        for genreID in genreIDs:
            if genreID in train_data.get(userID, {}):
                ratings.append(train_data[userID][genreID])

        # Calculate stats for the ratings
        stats = calculate_stats(ratings)

        # Combine the statistics to form a total score for the track
        total_score = sum(stats)

        # Decision threshold to classify as '1' or '0'
        threshold = 150  # Adjust based on validation
        prediction = 1 if total_score > threshold else 0

        # Write the result to the CSV
        csv_writer.writerow([kaggle_trackID, prediction])
